In [5]:
!pip install pyttsx3 pyaudio pywhatkit

  Using cached pyttsx3-2.90-py3-none-any.whl (39 kB)
  Obtaining dependency information for pyaudio from https://files.pythonhosted.org/packages/82/d8/f043c854aad450a76e476b0cf9cda1956419e1dacf1062eb9df3c0055abe/PyAudio-0.2.14-cp311-cp311-win_amd64.whl.metadata
  Using cached pywhatkit-5.4-py3-none-any.whl (15 kB)
  Obtaining dependency information for comtypes from https://files.pythonhosted.org/packages/c2/a7/fe4bd49b5c4afa7a7ed3852abda6909e48c00715e6a134e47055381113aa/comtypes-1.2.0-py2.py3-none-any.whl.metadata
  Using cached comtypes-1.2.0-py2.py3-none-any.whl.metadata (3.7 kB)
  Using cached pypiwin32-223-py3-none-any.whl (1.7 kB)
  Using cached PyAutoGUI-0.9.54.tar.gz (61 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finish

In [7]:
!pip install tensorflow spacy scikit-learn

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/90/f0/0133b684e18932c7bf4075d94819746cee2c0329f2569db526b0fa1df1df/spacy-3.7.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dep

In [1]:
#!python -m spacy download en_core_web_sm
!pip install speechrecognition

In [9]:
import tkinter as tk
from tkinter import Entry, Button, PhotoImage
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import spacy
import numpy as np
from tensorflow.keras.models import load_model
import threading

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load your neural network model (replace with your actual model)
model_filename = "chatbot.h5"
model_path = os.path.join("D:/downloads", model_filename)
nn_model = load_model(model_path)

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('800x600')
        self.root.resizable(False, False)
        self.message = tk.StringVar()

        self.textcon = tk.Text(self.root, bd=1, bg='white', width=50, height=8)
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=30, xscrollcommand=True, textvariable=self.message)
        self.mes_win.place(x=1, y=540, height=60, width=579)
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black')
        self.textcon.insert(tk.END, "NLPAssistant : This is Friday! Your Assistant.\n\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.button_send = Button(self.root, text='Send', bg='dark green', activebackground='grey',
                                  command=self.send_msz, width=12, height=5, font=('Arial'))
        self.button_send.place(x=690, y=540, height=60, width=110)

        mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(mic_image_path):
            self.mic_image = PhotoImage(file=mic_image_path).subsample(2, 3)
        else:
            print(f"Error: Mic image not found at {mic_image_path}")
            self.mic_image = None

        self.root.bind('<Return>', self.send_msz)
        self.root.mainloop()

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()
        self.textcon.insert(tk.END, f'You : {usr_input}' + '\n', 'usr')

        if usr_input.lower() == "end ":
            self.textcon.config(fg='black')
            self.textcon.insert(tk.END, "NLPAssistant : Thank You sir, I hope I assisted you properly \n")
            return self.root.destroy()

        else:
            self.textcon.config(fg='black')
            response = self.handle_user_input(usr_input)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            self.mes_win.delete(0, tk.END)

    def activate_mic_and_send(self):
        self.activate_mic()
        self.send_msz()

    def activate_mic(self):
        # Replace this with your logic for activating the microphone
        engine = pyttsx3.init()
        rate = engine.getProperty('rate')
        engine.setProperty('rate', 170)
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[1].id)
        r = sr.Recognizer()
        with sr.Microphone() as source:
            r.energy_threshold = 400
            r.adjust_for_ambient_noise(source, 1.2)
            print("Listening to You sir : ")
            audio = r.listen(source)
            text = r.recognize_google(audio)
            print("You Told : "+text)

            # Handle the user input (speech-to-text)
            response = self.handle_user_input(text)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data["intents"]:
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"Answer from JSON file: {answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [3]:
import tkinter as tk
from tkinter import Entry, Button, PhotoImage
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()

        self.textcon = tk.Text(self.root, bd=1, bg='white', width=70, height=15)  # Adjusted dimensions
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message)
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black')
        self.textcon.insert(tk.END, "NLPAssistant : This is Friday! Your Assistant.\n\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(mic_image_path):
            self.mic_image = PhotoImage(file=mic_image_path).subsample(2, 3)
        else:
            print(f"Error: Mic image not found at {mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, text='Mic', bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Arial'))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='dark green', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Arial'))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.root.mainloop()

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()
        self.textcon.insert(tk.END, f'You : {usr_input}' + '\n', 'usr')

        if usr_input.lower() in ["goodbye", "bye"]:
            self.textcon.config(fg='black')
            self.textcon.insert(tk.END, "NLPAssistant : Thank You sir, I hope I assisted you properly \n")
            return self.root.destroy()

        else:
            self.textcon.config(fg='black')
            response = self.handle_user_input(usr_input)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            self.mes_win.delete(0, tk.END)

    def activate_mic(self):
        if self.mic_image:
            self.mic_button.config(image=self.mic_image)
        else:
            self.mic_button.config(text="Mic")

        # Replace this with your logic for activating the microphone
        engine = pyttsx3.init()
        rate = engine.getProperty('rate')
        engine.setProperty('rate', 170)
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[1].id)
        r = sr.Recognizer()
        with sr.Microphone() as source:
            r.energy_threshold = 400
            r.adjust_for_ambient_noise(source, 1.2)
            self.textcon.insert(tk.END, "NLPAssistant: Listening... \n")  # Display listening message in textcon
            audio = r.listen(source)
            text = r.recognize_google(audio)
            self.textcon.insert(tk.END, f"You : {text}\n", 'usr')  # Display user's input in textcon

            # Handle the user input (speech-to-text)
            response = self.handle_user_input(text)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data["intents"]:
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"Answer from JSON file: {answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\anacoda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\aluru\AppData\Local\Temp\ipykernel_12868\3982504048.py", line 97, in activate_mic
    text = r.recognize_google(audio)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anacoda\Lib\site-packages\speech_recognition\recognizers\google.py", line 251, in recognize_legacy
    return output_parser.parse(response_text)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anacoda\Lib\site-packages\speech_recognition\recognizers\google.py", line 124, in parse
    actual_result = self.

In [2]:
import tkinter as tk
from tkinter import Entry, Button, PhotoImage
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()

        self.textcon = tk.Text(self.root, bd=1, bg='white', width=70, height=15)  # Adjusted dimensions
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message)
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black')
        self.textcon.insert(tk.END, "NLPAssistant : This is Friday! Your Assistant.\n\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(mic_image_path):
            self.mic_image = PhotoImage(file=mic_image_path).subsample(2, 3)
        else:
            print(f"Error: Mic image not found at {mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, text='Mic', bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Arial'))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='dark green', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Arial'))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.mainloop()

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()
        self.textcon.insert(tk.END, f'You : {usr_input}' + '\n', 'usr')

        if usr_input.lower() in ["goodbye", "bye"]:
            self.textcon.config(fg='black')
            response = "Thank You sir, I hope I assisted you properly"
            self.textcon.insert(tk.END, f"NLPAssistant : {response}\n")
            self.speak(response)
            return self.root.destroy()

        else:
            self.textcon.config(fg='black')
            response = self.handle_user_input(usr_input)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            self.speak(response)
            self.mes_win.delete(0, tk.END)

    def activate_mic(self):
        if self.mic_image:
            self.mic_button.config(image=self.mic_image)
        else:
            self.mic_button.config(text="Mic")

        engine = pyttsx3.init()
        rate = engine.getProperty('rate')
        engine.setProperty('rate', 170)
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[1].id)
        r = sr.Recognizer()
        with sr.Microphone() as source:
            r.energy_threshold = 400
            r.adjust_for_ambient_noise(source, 1.2)
            self.textcon.insert(tk.END, "NLPAssistant: Listening... \n")  # Display listening message in textcon
            audio = r.listen(source)
            text = r.recognize_google(audio)
            self.textcon.insert(tk.END, f"You : {text}\n", 'usr')  # Display user's input in textcon

            # Handle the user input (speech-to-text)
            response = self.handle_user_input(text)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            self.speak(response)

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [4]:
import tkinter as tk
from tkinter import Entry, Button, PhotoImage
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()

        self.textcon = tk.Text(self.root, bd=1, bg='white', width=70, height=15)  # Adjusted dimensions
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message)
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black')
        self.textcon.insert(tk.END, "NLPAssistant : This is Friday! Your Assistant.\n\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(mic_image_path):
            self.mic_image = PhotoImage(file=mic_image_path).subsample(2, 3)
        else:
            print(f"Error: Mic image not found at {mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, text='Mic', bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Arial'))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='dark green', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Arial'))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.mainloop()

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()
        self.textcon.insert(tk.END, f'You : {usr_input}' + '\n', 'usr')

        if usr_input.lower() in ["goodbye", "bye"]:
            self.textcon.config(fg='black')
            response = "Thank You sir, I hope I assisted you properly"
            self.textcon.insert(tk.END, f"NLPAssistant : {response}\n")
            self.speak(response)
            return self.root.destroy()

        else:
            self.textcon.config(fg='black')
            response = self.handle_user_input(usr_input)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            # Speak only if the input is from the microphone
            if self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def activate_mic(self):
        if self.mic_image:
            self.mic_button.config(image=self.mic_image)
        else:
            self.mic_button.config(text="Mic")

        engine = pyttsx3.init()
        rate = engine.getProperty('rate')
        engine.setProperty('rate', 170)
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[1].id)
        r = sr.Recognizer()
        with sr.Microphone() as source:
            r.energy_threshold = 400
            r.adjust_for_ambient_noise(source, 1.2)
            self.textcon.insert(tk.END, "NLPAssistant: Listening... \n")  # Display listening message in textcon
            audio = r.listen(source)
            text = r.recognize_google(audio)
            self.textcon.insert(tk.END, f"You (Mic): {text}\n", 'usr')  # Display user's input in textcon
            self.input_source = 'mic'

            # Handle the user input (speech-to-text)
            response = self.handle_user_input(text)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            self.speak(response)

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [2]:
#working code 1 
import tkinter as tk
from tkinter import Entry, Button, PhotoImage
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
import webbrowser

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()

        self.textcon = tk.Text(self.root, bd=1, bg='white', width=70, height=15)  # Adjusted dimensions
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message)
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black')
        self.textcon.insert(tk.END, "NLPAssistant : This is Friday! Your Assistant.\n\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(mic_image_path):
            self.mic_image = PhotoImage(file=mic_image_path).subsample(2, 3)
        else:
            print(f"Error: Mic image not found at {mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, text='Mic', bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Arial'))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='dark green', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Arial'))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.mainloop()

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()
        self.textcon.insert(tk.END, f'You : {usr_input}' + '\n', 'usr')

        if usr_input.lower() in ["goodbye", "bye"]:
            self.textcon.config(fg='black')
            response = "Thank You sir, I hope I assisted you properly"
            self.textcon.insert(tk.END, f"NLPAssistant : {response}\n")
            self.speak(response)
            return self.root.destroy()

        else:
            self.textcon.config(fg='black')
            response = self.handle_user_input(usr_input)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            # Speak only if the input is from the microphone
            if self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def activate_mic(self):
        if self.mic_image:
            self.mic_button.config(image=self.mic_image)
        else:
            self.mic_button.config(text="Mic")

        engine = pyttsx3.init()
        rate = engine.getProperty('rate')
        engine.setProperty('rate', 170)
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[1].id)
        r = sr.Recognizer()
        with sr.Microphone() as source:
            r.energy_threshold = 400
            r.adjust_for_ambient_noise(source, 1.2)
            self.textcon.insert(tk.END, "NLPAssistant: Listening... \n")  # Display listening message in textcon
            audio = r.listen(source)
            text = r.recognize_google(audio)
            self.textcon.insert(tk.END, f"You (Mic): {text}\n", 'usr')  # Display user's input in textcon
            self.input_source = 'mic'

            # Handle the user input (speech-to-text)
            response = self.handle_user_input(text)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            self.speak(response)

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        # Check if the user wants to open a specific website
        if "open" in user_keywords and any(keyword in user_keywords for keyword in ["youtube"]):
            self.open_website(user_input)
            return "Opening in YouTube..."

        # Check if the user wants to search on Wikipedia
        if "search" in user_keywords and "for" in user_keywords:
            self.search_wikipedia(user_input)
            return "Searching on Wikipedia..."

        # Continue with the existing logic for other responses
        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def open_website(self, user_input):
        # Extract the website name from the user input
        website_name = next((word for word in user_input.split() if word.lower() in ["youtube"]), None)

        if website_name:
            # Open the specified website in the default web browser
            url = f"https://www.{website_name}.com"
            webbrowser.open(url)

    def search_wikipedia(self, user_input):
        # Extract the search query from the user input
        search_query = user_input.split("for", 1)[-1].strip()

        if search_query:
            # Search the query on Wikipedia
            url = f"https://en.wikipedia.org/wiki/Special:Search?search={search_query}"
            webbrowser.open(url)

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [8]:
#working code 2 with updations
import tkinter as tk
from tkinter import Entry, Button, PhotoImage
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime

# Add the following import at the beginning of the file
from speech_recognition import UnknownValueError

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False

        self.textcon = tk.Text(self.root, bd=1, bg='white', width=70, height=15)  # Adjusted dimensions
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message)
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black')
        self.textcon.insert(tk.END, "NLPAssistant : This is Friday! Your Assistant.\n\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(mic_image_path):
            self.mic_image = PhotoImage(file=mic_image_path).subsample(2, 3)
        else:
            print(f"Error: Mic image not found at {mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, text='Mic', bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Arial'))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='dark green', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Arial'))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.mainloop()

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()
        self.textcon.insert(tk.END, f'You : {usr_input}' + '\n', 'usr')

        if usr_input.lower() in ["goodbye", "bye"]:
            self.textcon.config(fg='black')
            response = "Thank You sir, I hope I assisted you properly"
            self.textcon.insert(tk.END, f"NLPAssistant : {response}\n")
            self.speak(response)
            return self.root.destroy()

        else:
            self.textcon.config(fg='black')
            response = self.handle_user_input(usr_input)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def activate_mic(self):
        if self.mic_image:
            self.mic_button.config(image=self.mic_image)
        else:
            self.mic_button.config(text="Mic")

        self.is_listening = True
        threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            if not self.greeted:
                self.greeted = True
                self.speak("Hello sir! I am your Friday.")
                current_time = datetime.now().strftime("%I:%M %p")
                self.speak(f"The current time is {current_time}")

            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                self.textcon.insert(tk.END, "NLPAssistant: Listening... \n")  # Display listening message in textcon

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    self.textcon.insert(tk.END, f"You (Mic): {text}\n", 'usr')  # Display user's input in textcon
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
                    if self.input_source == 'mic':
                        self.speak(response)

                except UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Exception in thread Thread-30 (listen_continuously):
Traceback (most recent call last):
  File "D:\anacoda\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\anacoda\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\aluru\AppData\Local\Temp\ipykernel_11572\3933899887.py", line 125, in listen_continuously
  File "D:\anacoda\Lib\tkinter\__init__.py", line 3808, in insert
    self.tk.call((self._w, 'insert', index, chars) + args)
RuntimeError: main thread is not in main loop
Exception in thread Thread-33 (listen_continuously):
Traceback (most recent call last):
  File "D:\anacoda\Lib\threading.py", line 1038, 

NLPAssistant: Sorry, I could not understand the audio. Please try again.
NLPAssistant: Sorry, I could not understand the audio. Please try again.
NLPAssistant: Sorry, I could not understand the audio. Please try again.
NLPAssistant: Sorry, I could not understand the audio. Please try again.


Exception in thread Thread-36 (listen_continuously):
Traceback (most recent call last):
  File "D:\anacoda\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\anacoda\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\aluru\AppData\Local\Temp\ipykernel_11572\3933899887.py", line 120, in listen_continuously
  File "D:\anacoda\Lib\tkinter\__init__.py", line 3808, in insert
    self.tk.call((self._w, 'insert', index, chars) + args)
RuntimeError: main thread is not in main loop
Exception in thread Thread-41 (listen_continuously):
Traceback (most recent call last):
  File "D:\anacoda\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\anacoda\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\aluru\AppData\Local\Temp\ipykernel_11572\3933899887.py", line 125, in listen_continuously
  File "D:\anacoda\Lib\tkinter\__init__.py", line 3808, in insert
   

In [6]:
# with youtube and wiki
import tkinter as tk
from tkinter import Entry, Button, PhotoImage
import speech_recognition as sr
import pyttsx3
import os
import webbrowser  # Added import for webbrowser
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
from speech_recognition import UnknownValueError

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False

        self.textcon = tk.Text(self.root, bd=1, bg='white', width=70, height=15)  # Adjusted dimensions
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message)
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black')
        self.textcon.insert(tk.END, "NLPAssistant : This is Friday! Your Assistant.\n\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(mic_image_path):
            self.mic_image = PhotoImage(file=mic_image_path).subsample(2, 3)
        else:
            print(f"Error: Mic image not found at {mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, text='Mic', bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Arial'))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='dark green', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Arial'))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.mainloop()

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()
        self.textcon.insert(tk.END, f'You : {usr_input}' + '\n', 'usr')

        if usr_input.lower() in ["goodbye", "bye"]:
            self.textcon.config(fg='black')
            response = "Thank You sir, I hope I assisted you properly"
            self.textcon.insert(tk.END, f"NLPAssistant : {response}\n")
            self.speak(response)
            return self.root.destroy()

        else:
            self.textcon.config(fg='black')
            response = self.handle_user_input(usr_input)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def activate_mic(self):
        if self.mic_image:
            self.mic_button.config(image=self.mic_image)
        else:
            self.mic_button.config(text="Mic")

        self.is_listening = True
        threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            if not self.greeted:
                self.greeted = True
                self.speak("Hello sir! I am your Friday.")
                current_time = datetime.now().strftime("%I:%M %p")
                self.speak(f"The current time is {current_time}")

            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                self.textcon.insert(tk.END, "NLPAssistant: Listening... \n")  # Display listening message in textcon

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    self.textcon.insert(tk.END, f"You (Mic): {text}\n", 'usr')  # Display user's input in textcon
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
                    if self.input_source == 'mic':
                        self.speak(response)

                except UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        if "translate" in user_keywords:
            self.speak("Sure, I can help with translation. Please select a language option.")

            # Simulate language options (you can replace this with actual options from the API)
            language_options = {"1": "es", "2": "fr", "3": "de"}

            translation_options = "\n".join([f"{index}. {language}" for index, language in enumerate(language_options.values(), start=1)])
            self.speak(f"Available translation options:\n{translation_options}")

            selected_option = input("Enter the number of the desired language option: ")
            selected_language = language_options.get(selected_option)

            if selected_language:
                self.speak(f"Great! Selected language: {selected_language}. Now, please say something in that language.")
                
                # Simulate the translation response (you can replace this with the actual translation from the API)
                translated_text = f"Simulated translation from {selected_language} to English."

                self.speak("Here is the translation:")
                self.textcon.insert(tk.END, f"NLPAssistant: Translation: {translated_text}\n")

            else:
                self.speak("Invalid option. Please try again.")
        elif "play" in user_keywords:
            self.speak("Sure! Opening YouTube.")
            webbrowser.open("https://www.youtube.com/")  # Open YouTube in a web browser
            self.textcon.insert(tk.END, "NLPAssistant: Opening YouTube...\n")
        elif "search for" in user_keywords:
            search_query = user_keywords[user_keywords.index("search for") + 1:]
            search_query = " ".join(search_query)
            
            self.speak(f"Sure! Searching for {search_query} on Wikipedia.")
            wikipedia_url = f"https://en.wikipedia.org/wiki/{search_query.replace(' ', '_')}"
            webbrowser.open(wikipedia_url)  # Open Wikipedia with the specific search query
            self.textcon.insert(tk.END, f"NLPAssistant: Opening Wikipedia for {search_query}...\n")
        else:
            input_ids = tokenizer.encode(user_input, return_tensors="tf")
            output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                        temperature=0.7)
            bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

            # Adjust the response based on the user's technical ability
            technical_ability = "tech"  # You need to get this value from somewhere
            if technical_ability == "tech":
                # Customize response for technical users
                pass
            else:
                # Customize response for non-technical users
                pass

            candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
            relevancy_scores = self.score_relevancy(user_input, candidate_answers)

            # Store or update relevancy scores for future learning
            # Update your model based on user feedback and learning algorithm

            return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [4]:
# Another one with changes
import tkinter as tk
from tkinter import Entry, Button, PhotoImage
import speech_recognition as sr
import pyttsx3
import os
import webbrowser
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
from speech_recognition import UnknownValueError

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False

        self.textcon = tk.Text(self.root, bd=1, bg='white', width=70, height=15)  # Adjusted dimensions
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message)
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black')
        self.textcon.insert(tk.END, "NLPAssistant : This is Friday! Your Assistant.\n\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(mic_image_path):
            self.mic_image = PhotoImage(file=mic_image_path).subsample(2, 3)
        else:
            print(f"Error: Mic image not found at {mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, text='Mic', bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Arial'))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='dark green', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Arial'))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.mainloop()

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()
        self.textcon.insert(tk.END, f'You : {usr_input}' + '\n', 'usr')

        if usr_input.lower() in ["goodbye", "bye"]:
            self.textcon.config(fg='black')
            response = "Thank You sir, I hope I assisted you properly"
            self.textcon.insert(tk.END, f"NLPAssistant : {response}\n")
            self.speak(response)
            return self.root.destroy()

        else:
            self.textcon.config(fg='black')
            response = self.handle_user_input(usr_input)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def activate_mic(self):
        if self.mic_image:
            self.mic_button.config(image=self.mic_image)
        else:
            self.mic_button.config(text="Mic")

        self.is_listening = True
        threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            if not self.greeted:
                self.greeted = True
                self.speak("Hello sir! I am your Friday.")
                current_time = datetime.now().strftime("%I:%M %p")
                self.speak(f"The current time is {current_time}")

            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                self.textcon.insert(tk.END, "NLPAssistant: Listening... \n")  # Display listening message in textcon

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    self.textcon.insert(tk.END, f"You (Mic): {text}\n", 'usr')  # Display user's input in textcon
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
                    if self.input_source == 'mic':
                        self.speak(response)

                except UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        if "translate" in user_keywords:
            self.speak("Sure, I can help with translation. Please select a language option.")

            # Simulate language options (you can replace this with actual options from the API)
            language_options = {"1": "es", "2": "fr", "3": "de"}

            translation_options = "\n".join([f"{index}. {language}" for index, language in enumerate(language_options.values(), start=1)])
            self.speak(f"Available translation options:\n{translation_options}")

            selected_option = input("Enter the number of the desired language option: ")
            selected_language = language_options.get(selected_option)

            if selected_language:
                self.speak(f"Great! Selected language: {selected_language}. Now, please say something in that language.")
                
                # Simulate the translation response (you can replace this with the actual translation from the API)
                translated_text = f"Simulated translation from {selected_language} to English."

                self.speak("Here is the translation:")
                self.textcon.insert(tk.END, f"NLPAssistant: Translation: {translated_text}\n")

            else:
                self.speak("Invalid option. Please try again.")
        elif "play" in user_keywords:
            self.speak("Sure! Opening YouTube.")
            webbrowser.open("https://www.youtube.com/")  # Open YouTube in a web browser
            self.textcon.insert(tk.END, "NLPAssistant: Opening YouTube...\n")
        elif "search for" in user_keywords:
            search_query = user_keywords[-1]  # Get the last keyword after "search for"
            self.speak(f"Sure! Searching for {search_query} on Wikipedia.")
            
            # Open Wikipedia with the specific search query
            wikipedia_url = f"https://en.wikipedia.org/wiki/{search_query.replace(' ', '_')}"
            webbrowser.open(wikipedia_url)
            self.textcon.insert(tk.END, f"NLPAssistant: Opening Wikipedia for {search_query}...\n")

        else:
            input_ids = tokenizer.encode(user_input, return_tensors="tf")
            output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                        temperature=0.7)
            bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

            # Adjust the response based on the user's technical ability
            technical_ability = "tech"  # You need to get this value from somewhere
            if technical_ability == "tech":
                # Customize response for technical users
                pass
            else:
                # Customize response for non-technical users
                pass

            candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
            relevancy_scores = self.score_relevancy(user_input, candidate_answers)

            # Store or update relevancy scores for future learning
            # Update your model based on user feedback and learning algorithm

            return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Exception in thread Thread-20 (listen_continuously):
Traceback (most recent call last):
  File "D:\anacoda\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\anacoda\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\aluru\AppData\Local\Temp\ipykernel_14844\1773103816.py", line 129, in listen_continuously
  File "D:\anacoda\Lib\tkinter\__init__.py", line 3808, i

In [5]:
import tkinter as tk
from tkinter import Entry, Button, PhotoImage
import speech_recognition as sr
import pyttsx3
import os
import pywhatkit  # Import pywhatkit
import spacy
import threading
from datetime import datetime
from speech_recognition import UnknownValueError

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False

        self.textcon = tk.Text(self.root, bd=1, bg='white', width=70, height=15)  # Adjusted dimensions
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message)
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black')
        self.textcon.insert(tk.END, "NLPAssistant : This is Friday! Your Assistant.\n\n")

        mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(mic_image_path):
            self.mic_image = PhotoImage(file=mic_image_path).subsample(2, 3)
        else:
            print(f"Error: Mic image not found at {mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, text='Mic', bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Arial'))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='dark green', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Arial'))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.mainloop()

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()
        self.textcon.insert(tk.END, f'You : {usr_input}' + '\n', 'usr')

        if usr_input.lower() in ["goodbye", "bye"]:
            self.textcon.config(fg='black')
            response = "Thank You sir, I hope I assisted you properly"
            self.textcon.insert(tk.END, f"NLPAssistant : {response}\n")
            self.speak(response)
            return self.root.destroy()

        else:
            self.textcon.config(fg='black')
            response = self.handle_user_input(usr_input)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def activate_mic(self):
        if self.mic_image:
            self.mic_button.config(image=self.mic_image)
        else:
            self.mic_button.config(text="Mic")

        self.is_listening = True
        threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            if not self.greeted:
                self.greeted = True
                self.speak("Hello sir! I am your Friday.")
                current_time = datetime.now().strftime("%I:%M %p")
                self.speak(f"The current time is {current_time}")

            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                self.textcon.insert(tk.END, "NLPAssistant: Listening... \n")  # Display listening message in textcon

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    self.textcon.insert(tk.END, f"You (Mic): {text}\n", 'usr')  # Display user's input in textcon
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
                    if self.input_source == 'mic':
                        self.speak(response)

                except UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        if "search for" in user_keywords:
            search_query = user_keywords[-1]  # Get the last keyword after "search for"
            self.speak(f"Sure! Searching for {search_query} using pywhatkit.")
            
            # Use pywhatkit for searching
            pywhatkit.search(search_query)
            self.textcon.insert(tk.END, f"NLPAssistant: Searching for {search_query} using pywhatkit...\n")

        # Add other conditions for different types of queries
        
        else:
            # Add your logic for handling other types of queries
            pass

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


NLPAssistant: Sorry, I could not understand the audio. Please try again.
NLPAssistant: Sorry, I could not understand the audio. Please try again.
NLPAssistant: Sorry, I could not understand the audio. Please try again.


Exception in thread Thread-22 (listen_continuously):
Traceback (most recent call last):
  File "D:\anacoda\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\anacoda\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\aluru\AppData\Local\Temp\ipykernel_14844\2410033901.py", line 104, in listen_continuously
  File "D:\anacoda\Lib\tkinter\__init__.py", line 3808, in insert
    self.tk.call((self._w, 'insert', index, chars) + args)
RuntimeError: main thread is not in main loop


In [3]:
pip install webdriver_manager

  Obtaining dependency information for webdriver_manager from https://files.pythonhosted.org/packages/b1/51/b5c11cf739ac4eecde611794a0ec9df420d0239d51e73bc19eb44f02b48b/webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [2]:
#working code 2 with updations
import tkinter as tk
from tkinter import Entry, Button, PhotoImage
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager  # Make sure you have webdriver_manager installed

# Add the following import at the beginning of the file
from speech_recognition import UnknownValueError

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False

        self.textcon = tk.Text(self.root, bd=1, bg='white', width=70, height=15)  # Adjusted dimensions
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message)
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black')
        self.textcon.insert(tk.END, "NLPAssistant : This is Friday! Your Assistant.\n\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(mic_image_path):
            self.mic_image = PhotoImage(file=mic_image_path).subsample(2, 3)
        else:
            print(f"Error: Mic image not found at {mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, text='Mic', bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Arial'))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='dark green', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Arial'))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.mainloop()

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()
        self.textcon.insert(tk.END, f'You : {usr_input}' + '\n', 'usr')

        if usr_input.lower() in ["goodbye", "bye"]:
            self.textcon.config(fg='black')
            response = "Thank You sir, I hope I assisted you properly"
            self.textcon.insert(tk.END, f"NLPAssistant : {response}\n")
            self.speak(response)
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            self.textcon.config(fg='black')
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            self.speak(response)

        elif "search" in usr_input.lower():
            # Use Selenium to perform a search
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        else:
            self.textcon.config(fg='black')
            response = self.handle_user_input(usr_input)
            self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def perform_search(self, search_query):
        # Use Selenium to perform a search
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get("https://www.google.com/")
        search_box = driver.find_element_by_name("q")
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)
        self.textcon.config(fg='black')
        response = f"Performing a search for {search_query}..."
        self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
        self.speak(response)

    def activate_mic(self):
        if self.mic_image:
            self.mic_button.config(image=self.mic_image)
        else:
            self.mic_button.config(text="Mic")

        self.is_listening = True
        threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            if not self.greeted:
                self.greeted = True
                self.speak("Hello sir! I am your Friday.")
                current_time = datetime.now().strftime("%I:%M %p")
                self.speak(f"The current time is {current_time}")

            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                self.textcon.insert(tk.END, "NLPAssistant: Listening... \n")  # Display listening message in textcon

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    self.textcon.insert(tk.END, f"You (Mic): {text}\n", 'usr')  # Display user's input in textcon
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    self.textcon.insert(tk.END, f"NLPAssistant: {response}\n")
                    if self.input_source == 'mic':
                        self.speak(response)

                except UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\anacoda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\aluru\AppData\Local\Temp\ipykernel_3496\3290793408.py", line 98, in send_msz
    self.perform_search(search_query)
  File "C:\Users\aluru\AppData\Local\Temp\ipykernel_3496\3290793408.py", line 111, in perform_search
    driver = webdriver.Chrome(ChromeDriverManager().install())
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anacoda\Lib\site-packages\webdriver_manager\chrome.py", line 40, in install
    driver_path = self._get_driver_binary_path(self.dr

In [21]:
import tkinter as tk
from tkinter import Entry, Button, Label
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from speech_recognition import UnknownValueError

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False

        self.textcon = tk.Text(self.root, bd=1, bg='grey', width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right')  # Updated justify='right'
        self.textcon.tag_config('bot', foreground='black', justify='left')  # Updated justify='left'

        self.textcon.insert(tk.END, "NLPAssistant : This is Friday! Your Assistant.\n\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(self.mic_image_path):
            self.mic_image = tk.PhotoImage(file=self.mic_image_path).subsample(2, 3)
        else:
            print(f"Error: Mic image not found at {self.mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, text='Mic', bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 12))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='dark green', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 12))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.mainloop()

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()
        self.insert_message(f'You : {usr_input}', 'usr')

        if usr_input.lower() in ["goodbye", "bye"]:
            response = "Thank You sir, I hope I assisted you properly"
            self.insert_message(f"NLPAssistant : {response}", 'bot')
            self.speak(response)
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            self.insert_message(f"NLPAssistant: {response}", 'bot')
            self.speak(response)

        elif "search" in usr_input.lower():
            # Use Selenium to perform a search
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        else:
            response = self.handle_user_input(usr_input)
            self.insert_message(f"NLPAssistant: {response}", 'bot')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_message(self, message, tag):
        # Move the newline to the beginning
        self.textcon.insert(tk.END, '\n')

        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)

    def perform_search(self, search_query):
        # Use Selenium to perform a search
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get("https://www.google.com/")
        search_box = driver.find_element_by_name("q")
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)
        response = f"Performing a search for {search_query}..."
        self.insert_message(f"NLPAssistant: {response}", 'bot')
        self.speak(response)

    def activate_mic(self):
        if self.mic_image:
            self.mic_button.config(image=self.mic_image)
        else:
            self.mic_button.config(text="Mic")

        self.is_listening = True
        threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            if not self.greeted:
                self.greeted = True
                self.speak("Hello sir! I am your Friday.")
                current_time = datetime.now().strftime("%I:%M %p")
                self.speak(f"The current time is {current_time}")

            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                self.insert_message("NLPAssistant: Listening...", 'bot')  # Display listening message in textcon

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    self.insert_message(f"You (Mic): {text}", 'usr')  # Display user's input in textcon
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    self.insert_message(f"NLPAssistant: {response}", 'bot')
                    if self.input_source == 'mic':
                        self.speak(response)

                except UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [2]:
import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image, ImageTk

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False

        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right')  # Updated justify='right'
        self.textcon.tag_config('bot', foreground='black', justify='left')  # Updated justify='left'

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(self.mic_image_path):
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        else:
            print(f"Error: Mic image not found at {self.mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 12))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 12))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am your Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)

        elif "search" in usr_input.lower():
            # Use Selenium to perform a search
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")  # Add a newline after each message
        self.textcon.yview(tk.END)

    def perform_search(self, search_query):
        # Use Selenium to perform a search
        driver = webdriver.Edge(EdgeDriverManager().install())
        driver.get("https://www.google.com/")
        search_box = driver.find_element_by_name("q")
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)
        response = f"Performing a search for {search_query}..."
        # Insert bot icon and name
        self.insert_user('bot', 'NLPAssistant', self.bot_icon)
        # Insert bot response
        self.insert_message('bot', f'{response}')
        self.speak(response)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                # Insert bot icon and name
                self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    # Insert user icon and name
                    self.insert_user('usr', 'You', self.user_icon)
                    # Insert user input
                    self.insert_message('usr', f'{text}')
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    # Insert bot icon and name
                    self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                    # Insert bot response
                    self.insert_message('bot', f'{response}')
                    if self.input_source == 'mic':
                        self.speak(response)

                except sr.UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
C:\Users\aluru\AppData\Local\Temp\ipykernel_16028\1614564686.py:58: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_16028\1614564686.py:59: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_16028\1614564686.py:64: Deprecat

In [31]:
# scrolling issue fixed

import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image, ImageTk

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False
        

        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right')
        self.textcon.tag_config('bot', foreground='black', justify='left')

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"
        if os.path.exists(self.mic_image_path):
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        else:
            print(f"Error: Mic image not found at {self.mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 12))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 12))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)
        

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am your Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)

        elif "search" in usr_input.lower():
            # Use Selenium to perform a search
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")  # Add a newline after each message
        self.textcon.see(tk.END)

    def perform_search(self, search_query):
        # Use Selenium to perform a search
        driver = webdriver.Edge(EdgeDriverManager().install())
        driver.get("https://www.google.com/")
        search_box = driver.find_element_by_name("q")
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)
        response = f"Performing a search for {search_query}..."
        # Insert bot icon and name
        self.insert_user('bot', 'NLPAssistant', self.bot_icon)
        # Insert bot response
        self.insert_message('bot', f'{response}')
        self.speak(response)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                # Insert bot icon and name
                self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    # Insert user icon and name
                    self.insert_user('usr', 'You', self.user_icon)
                    # Insert user input
                    self.insert_message('usr', f'{text}')
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    # Insert bot icon and name
                    self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                    # Insert bot response
                    self.insert_message('bot', f'{response}')
                    if self.input_source == 'mic':
                        self.speak(response)

                except sr.UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
C:\Users\aluru\AppData\Local\Temp\ipykernel_15312\3072366810.py:61: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_15312\3072366810.py:62: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_15312\3072366810.py:67: Deprecat

TclError: image "pyimage60" doesn't exist

In [11]:
import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image, ImageTk
from googletrans import Translator

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False
        self.target_language = 'en'  # Default language

        self.setup_ui()

        # Load spaCy model for English
        self.nlp = spacy.load("en_core_web_sm")

        # Load pre-trained language model (GPT-2)
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

        # Load or collect data for training/fine-tuning (replace with your actual data loading)
        with open('D:/downloads/tech.json', 'r') as file:
            self.data = json.load(file)

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def setup_ui(self):
        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right')
        self.textcon.tag_config('bot', foreground='black', justify='left')

        self.textcon.insert(tk.END, "\n")

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"
        self.mic_image_path = "D:/downloads/mic.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.LANCZOS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.LANCZOS))

        if os.path.exists(self.mic_image_path):
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.LANCZOS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        else:
            print(f"Error: Mic image not found at {self.mic_image_path}")
            self.mic_image = None

        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 12))
        self.mic_button.place(x=800, y=720, height=60, width=110)

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 12))
        self.send_button.place(x=680, y=720, height=60, width=110)

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            self.insert_message('bot', f"The current time is {current_time}")
            self.speak("Hello sir! I am your Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        self.insert_user('usr', 'You', self.user_icon)
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False
            return self.root.destroy()

        elif "play" in usr_input.lower():
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            self.insert_message('bot', f'{response}')
            self.speak(response)

        elif "search" in usr_input.lower():
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        else:
            response = self.handle_user_input(usr_input)
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            self.insert_message('bot', f'{response}')
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        self.textcon.image_create(tk.END, image=icon)
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")
        self.textcon.see(tk.END)

    def perform_search(self, search_query):
        driver = webdriver.Edge(ChromeDriverManager().install())
        driver.get("https://www.google.com/")
        search_box = driver.find_element_by_name("q")
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)
        response = f"Performing a search for {search_query}..."
        self.insert_user('bot', 'NLPAssistant', self.bot_icon)
        self.insert_message('bot', f'{response}')
        self.speak(response)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    self.insert_user('usr', 'You', self.user_icon)
                    self.insert_message('usr', f'{text}')
                    self.input_source = 'mic'
                    response = self.handle_user_input(text)
                    self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                    self.insert_message('bot', f'{response}')
                    if self.input_source == 'mic':
                        self.speak(response)

                except sr.UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in self.data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = self.tokenizer.encode(user_input, return_tensors="tf")
        output = self.gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
        bot_response = self.tokenizer.decode(output[0], skip_special_tokens=True)

        if "translate" in user_input.lower():
            lang_input = user_input.lower().replace("translate", "").strip()
            self.set_target_language(lang_input)
            response = f"Language set to {self.target_language.upper()}. Please continue speaking in {self.target_language.upper()}."
        else:
            translated_input = self.translate_to_target_language(user_input)
            self.insert_user('usr', 'You', self.user_icon)
            self.insert_message('usr', f'{self.target_language.upper()}: {translated_input}')
            response = self.check_json_answers(translated_input)

            if not response:
                response = self.generate_gpt2_response(translated_input)

        self.insert_user('bot', 'NLPAssistant', self.bot_icon)
        self.insert_message('bot', f'{self.target_language.upper()}: {response}')

        if hasattr(self, 'input_source') and self.input_source == 'mic':
            self.speak(response)
        self.mes_win.delete(0, tk.END)

    def set_target_language(self, language):
        self.target_language = language

    def translate_to_target_language(self, text):
        translator = Translator()
        translated_text = translator.translate(text, dest=self.target_language).text
        return translated_text

    def check_json_answers(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in self.data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return answer

        return None

    def generate_gpt2_response(self, user_input):
        input_ids = self.tokenizer.encode(user_input, return_tensors="tf")
        output = self.gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
        bot_response = self.tokenizer.decode(output[0], skip_special_tokens=True)

        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        return bot_response

    def extract_keywords(self, text):
        doc = self.nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
D:\anacoda\Lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
D:\anacoda\Lib\site-packages\transformers\generation\configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavi

In [6]:
pip install openai==0.28.0


  Obtaining dependency information for openai==0.28.0 from https://files.pythonhosted.org/packages/ae/59/911d6e5f1d7514d79c527067643376cddcf4cb8d1728e599b3b03ab51c69/openai-0.28.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/76.5 kB ? eta -:--:--
   -------------------------- ------------- 51.2/76.5 kB 650.2 kB/s eta 0:00:01
   ---------------------------------------- 76.5/76.5 kB 705.4 kB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.6.1
    Uninstalling openai-1.6.1:
      Successfully uninstalled openai-1.6.1


In [3]:
!pip install --upgrade httpcore==0.17.4
!pip install --upgrade httpx==0.18.2

ERROR: Could not find a version that satisfies the requirement httpcore==0.17.4 (from versions: 0.0.1, 0.0.2, 0.0.3, 0.1.0, 0.1.1, 0.2.0, 0.2.1, 0.3.0, 0.4.0, 0.5.0, 0.6.0, 0.7.0, 0.8.0, 0.8.1, 0.8.2, 0.8.3, 0.8.4, 0.9.0, 0.9.1, 0.10.0, 0.10.1, 0.10.2, 0.11.0, 0.11.1, 0.12.0, 0.12.1, 0.12.2, 0.12.3, 0.13.0, 0.13.1, 0.13.2, 0.13.3, 0.13.4, 0.13.5, 0.13.6, 0.13.7, 0.14.0, 0.14.1, 0.14.2, 0.14.3, 0.14.4, 0.14.5, 0.14.6, 0.14.7, 0.15.0, 0.16.0, 0.16.1, 0.16.2, 0.16.3, 0.17.0, 0.17.1, 0.17.2, 0.17.3, 0.18.0, 1.0.0, 1.0.1, 1.0.2)
ERROR: No matching distribution found for httpcore==0.17.4


In [10]:
pip install google

     ---------------------------------------- 0.0/45.3 kB ? eta -:--:--
     --------- ------------------------------ 10.2/45.3 kB ? eta -:--:--
     ----------------- -------------------- 20.5/45.3 kB 162.5 kB/s eta 0:00:01
     ----------------- -------------------- 20.5/45.3 kB 162.5 kB/s eta 0:00:01
     ----------------- -------------------- 20.5/45.3 kB 162.5 kB/s eta 0:00:01
     ----------------- -------------------- 20.5/45.3 kB 162.5 kB/s eta 0:00:01
     ------------------------- ------------ 30.7/45.3 kB 100.9 kB/s eta 0:00:01
     -------------------------------------- 45.3/45.3 kB 124.6 kB/s eta 0:00:00

  Using cached google-3.0.0-py2.py3-none-any.whl (45 kB)


In [17]:
# scrolling issue fixed, utube and search is working

import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from googlesearch import search
import webbrowser
from PIL import Image, ImageTk

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False
        

        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right')
        self.textcon.tag_config('bot', foreground='black', justify='left')

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"

        try:
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        except Exception as e:
            print(f"Error loading mic image: {e}")
            self.mic_image = None


        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 12))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 12))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)
        

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am your Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.mes_win.delete(0, tk.END)

        elif "search" in usr_input.lower():
            # Use Selenium to perform a search
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")  
        self.textcon.see(tk.END)
        
    def perform_search(self, search_query):
        try:
            search_results = search(search_query, num=1, stop=1, pause=2)
            first_result = next(search_results)

            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant: here is the link', self.bot_icon)
            self.textcon.tag_config('link', foreground='blue', underline=True)
            self.textcon.insert(tk.END, f'{first_result}\n', 'link')
            self.textcon.tag_bind('link', '<Button-1>', lambda event, link=first_result: self.open_link(link))
            self.textcon.insert(tk.END, "\n")
        except StopIteration:
            response = f"Sorry, I couldn't find information about {search_query}."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}\n')

        self.mes_win.delete(0, tk.END)

    def open_link(self, link):
        webbrowser.open(link)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                # Insert bot icon and name
                self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    # Insert user icon and name
                    self.insert_user('usr', 'You', self.user_icon)
                    # Insert user input
                    self.insert_message('usr', f'{text}')
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    # Insert bot icon and name
                    self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                    # Insert bot response
                    self.insert_message('bot', f'{response}')
                    if self.input_source == 'mic':
                        self.speak(response)

                except sr.UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
C:\Users\aluru\AppData\Local\Temp\ipykernel_5768\3406253894.py:60: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_5768\3406253894.py:61: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_5768\3406253894.py:67: Deprecation

In [1]:
# scrolling issue fixed, utube and search is working

import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from googlesearch import search
import webbrowser
from PIL import Image, ImageTk

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False
        

        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right')
        self.textcon.tag_config('bot', foreground='black', justify='left')

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"

        try:
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        except Exception as e:
            print(f"Error loading mic image: {e}")
            self.mic_image = None


        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 12))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 12))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)
        

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.mes_win.delete(0, tk.END)

        elif "search" in usr_input.lower():
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")  
        self.textcon.see(tk.END)
        
    def perform_search(self, search_query):
        try:
            search_results = search(search_query, num=1, stop=1, pause=2)
            first_result = next(search_results)

            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant: here is the link', self.bot_icon)
            self.textcon.tag_config('link', foreground='blue', underline=True)
            self.textcon.insert(tk.END, f'{first_result}\n', 'link')
            self.textcon.tag_bind('link', '<Button-1>', lambda event, link=first_result: self.open_link(link))
            self.textcon.insert(tk.END, "\n")
        except StopIteration:
            response = f"Sorry, I couldn't find information about {search_query}."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}\n')

        self.mes_win.delete(0, tk.END)

    def open_link(self, link):
        webbrowser.open(link)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                # Insert bot icon and name
                self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    # Insert user icon and name
                    self.insert_user('usr', 'You', self.user_icon)
                    # Insert user input
                    self.insert_message('usr', f'{text}')
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    # Insert bot icon and name
                    self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                    # Insert bot response
                    self.insert_message('bot', f'{response}')
                    if self.input_source == 'mic':
                        self.speak(response)

                except sr.UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
C:\Users\aluru\AppData\Local\Temp\ipykernel_4380\4160261105.py:60: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_4380\4160261105.py:61: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_4380\4160261105.py:67: Deprecation

NLPAssistant: Sorry, I could not understand the audio. Please try again.


In [4]:
pip install google_trans_new

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from googlesearch import search
import webbrowser
from PIL import Image, ImageTk
from googletrans import Translator
from gtts import gTTS
import pygame
import google_trans_new

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False
        
        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right')
        self.textcon.tag_config('bot', foreground='black', justify='left')

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"

        try:
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        except Exception as e:
            print(f"Error loading mic image: {e}")
            self.mic_image = None

        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 12))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 12))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.mes_win.delete(0, tk.END)

        elif "search" in usr_input.lower():
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        elif "translate" in usr_input or "language" in usr_input:
            self.trans()

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")  
        self.textcon.see(tk.END)
        
    def perform_search(self, search_query):
        try:
            search_results = search(search_query, num=1, stop=1, pause=2)
            first_result = next(search_results)

            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant: here is the link', self.bot_icon)
            self.textcon.tag_config('link', foreground='blue', underline=True)
            self.textcon.insert(tk.END, f'{first_result}\n', 'link')
            self.textcon.tag_bind('link', '<Button-1>', lambda event, link=first_result: self.open_link(link))
            self.textcon.insert(tk.END, "\n")
        except StopIteration:
            response = f"Sorry, I couldn't find information about {search_query}."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}\n')

        self.mes_win.delete(0, tk.END)

    def open_link(self, link):
        webbrowser.open(link)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                # Insert bot icon and name
                self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    # Insert user icon and name
                    self.insert_user('usr', 'You', self.user_icon)
                    # Insert user input
                    self.insert_message('usr', f'{text}')
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    # Insert bot icon and name
                    self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                    # Insert bot response
                    self.insert_message('bot', f'{response}')
                    if self.input_source == 'mic':
                        self.speak(response)

                except sr.UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def trans(self):
        print(google_trans_new.LANGUAGES)

        mic = sr.Microphone()
        r = sr.Recognizer()

        with mic as source:
            translator = Translator()
            input_language = input("Select Language from: ")
            output_language = input("Select your Language To: ")

            print("Speak the text that needs to be translated ..")
            r.adjust_for_ambient_noise(source, duration=0.2)
            audio3 = r.listen(source)
            text3 = r.recognize_google(audio3)

            print("You said: " + text3)
            trans_to = translator.translate(text3, src=input_language, dest=output_language)
            transed_text = trans_to.text
            print("The translated meaning is: " + transed_text)
            speak = gTTS(text=transed_text, lang=output_language, slow=False)
            speak.save("captured_voice.mp3")
            pygame.mixer.init()
            pygame.mixer.music.load('captured_voice.mp3')
            pygame.mixer.music.play()

            print(transed_text)

    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
C:\Users\aluru\AppData\Local\Temp\ipykernel_13372\1437830125.py:61: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_13372\1437830125.py:62: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_13372\1437830125.py:68: Deprecat

AttributeError: 'ChatbotApp' object has no attribute 'translate_text'

In [1]:
import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from googlesearch import search
import webbrowser
from PIL import Image, ImageTk
from googletrans import Translator
from gtts import gTTS
import pygame
import google_trans_new

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False
        
        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right')
        self.textcon.tag_config('bot', foreground='black', justify='left')

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"

        try:
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        except Exception as e:
            print(f"Error loading mic image: {e}")
            self.mic_image = None

        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 12))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 12))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.mes_win.delete(0, tk.END)

        elif "search" in usr_input.lower():
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        elif "translate" in usr_input or "language" in usr_input:
            self.trans()

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")  
        self.textcon.see(tk.END)
        
    def perform_search(self, search_query):
        try:
            search_results = search(search_query, num=1, stop=1, pause=2)
            first_result = next(search_results)

            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant: here is the link', self.bot_icon)
            self.textcon.tag_config('link', foreground='blue', underline=True)
            self.textcon.insert(tk.END, f'{first_result}\n', 'link')
            self.textcon.tag_bind('link', '<Button-1>', lambda event, link=first_result: self.open_link(link))
            self.textcon.insert(tk.END, "\n")
        except StopIteration:
            response = f"Sorry, I couldn't find information about {search_query}."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}\n')

        self.mes_win.delete(0, tk.END)

    def open_link(self, link):
        webbrowser.open(link)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                # Insert bot icon and name
                self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    # Insert user icon and name
                    self.insert_user('usr', 'You', self.user_icon)
                    # Insert user input
                    self.insert_message('usr', f'{text}')
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    # Insert bot icon and name
                    self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                    # Insert bot response
                    self.insert_message('bot', f'{response}')
                    if self.input_source == 'mic':
                        self.speak(response)

                except sr.UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def trans(self):
        print(google_trans_new.LANGUAGES)

        mic = sr.Microphone()
        r = sr.Recognizer()

        with mic as source:
            translator = Translator()
            input_language = input("Select Language from: ")
            output_language = input("Select your Language To: ")

            print("Speak the text that needs to be translated ..")
            r.adjust_for_ambient_noise(source, duration=0.2)
            audio3 = r.listen(source)
            text3 = r.recognize_google(audio3)

            print("You said: " + text3)
            trans_to = translator.translate(text3, src=input_language, dest=output_language)
            transed_text = trans_to.text
            print("The translated meaning is: " + transed_text)

            # Update GUI with the translation
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            self.insert_message('bot', f'Translation: {transed_text}')
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                speak = gTTS(text=transed_text, lang=output_language, slow=False)
                speak.save("captured_voice.mp3")
                pygame.mixer.init()
                pygame.mixer.music.load('captured_voice.mp3')
                pygame.mixer.music.play()

            print(transed_text)

    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()



pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html



All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
C:\Users\aluru\AppData\Local\Temp\ipykernel_14376\496450202.py:61: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_14376\496450202.py:62: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_14376\496450202.py:68: Deprecation

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [24]:
import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from googlesearch import search
import webbrowser
from PIL import Image, ImageTk
from googletrans import Translator
from gtts import gTTS
import pygame
import google_trans_new
from tkinter import simpledialog

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False
        
        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right')
        self.textcon.tag_config('bot', foreground='black', justify='left')

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"

        try:
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        except Exception as e:
            print(f"Error loading mic image: {e}")
            self.mic_image = None

        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 12))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 12))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.mes_win.delete(0, tk.END)

        elif "search" in usr_input.lower():
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        elif "translate" in usr_input or "language" in usr_input:
            self.trans()
            self.mes_win.delete(0, tk.END)

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")  
        self.textcon.see(tk.END)
        
    def perform_search(self, search_query):
        try:
            search_results = search(search_query, num=1, stop=1, pause=2)
            first_result = next(search_results)

            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant: here is the link', self.bot_icon)
            self.textcon.tag_config('link', foreground='blue', underline=True)
            self.textcon.insert(tk.END, f'{first_result}\n', 'link')
            self.textcon.tag_bind('link', '<Button-1>', lambda event, link=first_result: self.open_link(link))
            self.textcon.insert(tk.END, "\n")
        except StopIteration:
            response = f"Sorry, I couldn't find information about {search_query}."
            # Insert bot icon and name
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}\n')

        self.mes_win.delete(0, tk.END)

    def open_link(self, link):
        webbrowser.open(link)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                # Insert bot icon and name
                self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    # Insert user icon and name
                    self.insert_user('usr', 'You', self.user_icon)
                    # Insert user input
                    self.insert_message('usr', f'{text}')
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    # Insert bot icon and name
                    self.insert_user('bot', 'NLPAssistant', self.bot_icon)
                    # Insert bot response
                    self.insert_message('bot', f'{response}')
                    if self.input_source == 'mic':
                        self.speak(response)

                except sr.UnknownValueError:
                    print("NLPAssistant: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"NLPAssistant: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def trans(self):
        languages = google_trans_new.LANGUAGES

        # Insert bot icon and name
        self.insert_user('bot', 'NLPAssistant', self.bot_icon)
        # Insert bot response
        self.insert_message('bot', 'Available Languages:')
        # Insert available languages in the GUI
        for code, lang in languages.items():
            self.insert_message('bot', f'{code}: {lang}')

        mic = sr.Microphone()
        r = sr.Recognizer()

        with mic as source:
            translator = Translator()
            
            # Get input language from user using simpledialog
            input_language = simpledialog.askstring("Input Language", "Select Language from:")
            # Get output language from user using simpledialog
            output_language = simpledialog.askstring("Output Language", "Select your Language To:")

            print("Speak the text that needs to be translated ..")
            r.adjust_for_ambient_noise(source, duration=0.2)
            audio3 = r.listen(source)
            text3 = r.recognize_google(audio3)

            print("You said: " + text3)
            trans_to = translator.translate(text3, src=input_language, dest=output_language)
            transed_text = trans_to.text

            # Update GUI with the translation
            self.insert_user('bot', 'NLPAssistant', self.bot_icon)
            self.insert_message('bot', f'Translation: {transed_text}')
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(transed_text)
            self.textcon.insert(tk.END, "\n")

    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
C:\Users\aluru\AppData\Local\Temp\ipykernel_14376\183394825.py:62: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_14376\183394825.py:63: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_14376\183394825.py:69: Deprecation

In [11]:
# full translation not working
import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from googlesearch import search
import webbrowser
from PIL import Image, ImageTk
from googletrans import Translator
from gtts import gTTS
import pygame
import google_trans_new
from tkinter import simpledialog

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False
        
        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right',font=('Times New Roman', 14, 'bold'), background='lightgreen')
        self.textcon.tag_config('bot', foreground='black', justify='left',font=('Times New Roman', 14, 'bold'), background='lightblue')

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"

        try:
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        except Exception as e:
            print(f"Error loading mic image: {e}")
            self.mic_image = None

        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 14))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 14, "bold"))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.mes_win.delete(0, tk.END)

        elif "search" in usr_input.lower():
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        elif "translate" in usr_input or "language" in usr_input:
            self.trans()
            self.mes_win.delete(0, tk.END)

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")  
        self.textcon.see(tk.END)
        
    def perform_search(self, search_query):
        try:
            search_results = search(search_query, num=1, stop=1, pause=2)
            first_result = next(search_results)

            # Insert bot icon and name
            self.insert_user('bot', 'Friday: here is the link', self.bot_icon)
            self.textcon.tag_config('link', foreground='blue', underline=True)
            self.textcon.insert(tk.END, f'{first_result}\n', 'link')
            self.textcon.tag_bind('link', '<Button-1>', lambda event, link=first_result: self.open_link(link))
            self.textcon.insert(tk.END, "\n")
        except StopIteration:
            response = f"Sorry, I couldn't find information about {search_query}."
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}\n')

        self.mes_win.delete(0, tk.END)

    def open_link(self, link):
        webbrowser.open(link)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                # Insert bot icon and name
                self.insert_user('bot', 'Friday', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    # Insert user icon and name
                    self.insert_user('usr', 'You', self.user_icon)
                    # Insert user input
                    self.insert_message('usr', f'{text}')
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    # Insert bot icon and name
                    self.insert_user('bot', 'Friday', self.bot_icon)
                    # Insert bot response
                    self.insert_message('bot', f'{response}')
                    if self.input_source == 'mic':
                        self.speak(response)

                except sr.UnknownValueError:
                    print("Friday: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"Friday: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def trans(self):
        languages = google_trans_new.LANGUAGES

        # Insert bot icon and name
        self.insert_user('bot', 'Friday', self.bot_icon)

        # Check if the translation process is already ongoing
        if not hasattr(self, 'translation_in_progress'):
            # Insert bot response
            self.insert_message('bot', 'Available Languages:')
            # Insert available languages in the GUI
            for code, lang in languages.items():
                self.insert_message('bot', f'{code}: {lang}')

            mic = sr.Microphone()
            r = sr.Recognizer()

            with mic as source:
                translator = Translator()

                # Get input language from user using simpledialog
                input_language = simpledialog.askstring("Input Language", "Select Language from:")
                # Get output language from user using simpledialog
                output_language = simpledialog.askstring("Output Language", "Select your Language To:")

                print("Speak the text that needs to be translated ..")
                r.adjust_for_ambient_noise(source, duration=0.2)
                audio3 = r.listen(source)
                text3 = r.recognize_google(audio3)

                print("You said: " + text3.lower())

                # Translate the user's question
                translated_question = translator.translate(text3, src=input_language, dest=output_language).text.lower()

                # Update GUI with the translation
                self.insert_user('bot', 'Friday', self.bot_icon)
                self.insert_message('bot', f'Translation: {translated_question}')
                if hasattr(self, 'input_source') and self.input_source == 'mic':
                    self.speak(translated_question)
                self.textcon.insert(tk.END, "\n")

                # Set the variable to indicate that translation is in progress
                self.translation_in_progress = True
                return  # Exit the function to avoid rerunning the translation process

        # If translation process is already in progress, clear the variable
        self.translation_in_progress = False


    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
C:\Users\aluru\AppData\Local\Temp\ipykernel_13612\881283532.py:63: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_13612\881283532.py:64: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_13612\881283532.py:70: Deprecation

Speak the text that needs to be translated ..
You said: how are you


In [5]:
# full translation not working
import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from googlesearch import search
import webbrowser
from PIL import Image, ImageTk
from googletrans import Translator
from gtts import gTTS
import pygame
import google_trans_new
from tkinter import simpledialog

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False
        
        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right',font=('Times New Roman', 14, 'bold'), background='lightgreen')
        self.textcon.tag_config('bot', foreground='black', justify='left',font=('Times New Roman', 14, 'bold'), background='lightblue')

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"

        try:
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        except Exception as e:
            print(f"Error loading mic image: {e}")
            self.mic_image = None

        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 14))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 14, "bold"))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.mes_win.delete(0, tk.END)

        elif "search" in usr_input.lower():
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        elif "translate" in usr_input or "language" in usr_input:
            self.trans()
            self.mes_win.delete(0, tk.END)

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")  
        self.textcon.see(tk.END)
        
    def perform_search(self, search_query):
        try:
            search_results = search(search_query, num=1, stop=1, pause=2)
            first_result = next(search_results)

            # Insert bot icon and name
            self.insert_user('bot', 'Friday: here is the link', self.bot_icon)
            self.textcon.tag_config('link', foreground='blue', underline=True)
            self.textcon.insert(tk.END, f'{first_result}\n', 'link')
            self.textcon.tag_bind('link', '<Button-1>', lambda event, link=first_result: self.open_link(link))
            self.textcon.insert(tk.END, "\n")
        except StopIteration:
            response = f"Sorry, I couldn't find information about {search_query}."
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}\n')

        self.mes_win.delete(0, tk.END)

    def open_link(self, link):
        webbrowser.open(link)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)
                # Insert bot icon and name
                self.insert_user('bot', 'Friday', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)
                    # Insert user icon and name
                    self.insert_user('usr', 'You', self.user_icon)
                    # Insert user input
                    self.insert_message('usr', f'{text}')
                    self.input_source = 'mic'

                    # Handle the user input (speech-to-text)
                    response = self.handle_user_input(text)
                    # Insert bot icon and name
                    self.insert_user('bot', 'Friday', self.bot_icon)
                    # Insert bot response
                    self.insert_message('bot', f'{response}')
                    if self.input_source == 'mic':
                        self.speak(response)

                except sr.UnknownValueError:
                    print("Friday: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"Friday: There was an error with the speech recognition service: {e}")

    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def trans(self):
        languages = google_trans_new.LANGUAGES

        # Insert bot icon and name
        self.insert_user('bot', 'Friday', self.bot_icon)

        # Check if the translation process is already ongoing
        if not hasattr(self, 'translation_in_progress'):
            # Insert bot response
            self.insert_message('bot', 'Available Languages:')
            # Insert available languages in the GUI
            for code, lang in languages.items():
                self.insert_message('bot', f'{code}: {lang}')

            mic = sr.Microphone()
            r = sr.Recognizer()

            with mic as source:
                translator = Translator()

                # Get input language from user using simpledialog
                input_language = simpledialog.askstring("Input Language", "Select Language from:")
                # Get output language from user using simpledialog
                output_language = simpledialog.askstring("Output Language", "Select your Language To:")

                self.insert_user('bot', 'Friday', self.bot_icon)
                self.insert_message('bot', 'speak the text you need to be translated..')
                r.adjust_for_ambient_noise(source, duration=0.2)
                audio3 = r.listen(source)
                text3 = r.recognize_google(audio3)
                
                self.insert_user('usr', 'You', self.user_icon)
                self.insert_message('usr',text3.lower())

                # Translate the user's question
                translated_question = translator.translate(text3, src=input_language, dest=output_language).text.lower()

                # Update GUI with the translation
                self.insert_user('bot', 'Friday', self.bot_icon)
                self.insert_message('bot', f'Translation: {translated_question}')
                if hasattr(self, 'input_source') and self.input_source == 'mic':
                    self.speak(translated_question)
                self.textcon.insert(tk.END, "\n")

                # Set the variable to indicate that translation is in progress
                self.translation_in_progress = True
                return  # Exit the function to avoid rerunning the translation process

        # If translation process is already in progress, clear the variable
        self.translation_in_progress = False


    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
C:\Users\aluru\AppData\Local\Temp\ipykernel_14140\1732724586.py:63: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_14140\1732724586.py:64: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_14140\1732724586.py:70: Deprecat

In [8]:
# full translation not working
import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from googlesearch import search
import webbrowser
from PIL import Image, ImageTk
from googletrans import Translator
from gtts import gTTS
import pygame
import google_trans_new
from tkinter import simpledialog

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False
        
        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right',font=('Times New Roman', 14, 'bold'), background='lightgreen')
        self.textcon.tag_config('bot', foreground='black', justify='left',font=('Times New Roman', 14, 'bold'), background='lightblue')

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"

        try:
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        except Exception as e:
            print(f"Error loading mic image: {e}")
            self.mic_image = None

        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 14))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 14, "bold"))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.mes_win.delete(0, tk.END)

        elif "search" in usr_input.lower():
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        elif "translate" in usr_input or "language" in usr_input:
            self.trans()
            self.mes_win.delete(0, tk.END)

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n")  
        self.textcon.see(tk.END)
        
    def perform_search(self, search_query):
        try:
            search_results = search(search_query, num=1, stop=1, pause=2)
            first_result = next(search_results)

            # Insert bot icon and name
            self.insert_user('bot', 'Friday: here is the link', self.bot_icon)
            self.textcon.tag_config('link', foreground='blue', underline=True)
            self.textcon.insert(tk.END, f'{first_result}\n', 'link')
            self.textcon.tag_bind('link', '<Button-1>', lambda event, link=first_result: self.open_link(link))
            self.textcon.insert(tk.END, "\n")
        except StopIteration:
            response = f"Sorry, I couldn't find information about {search_query}."
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}\n')

        self.mes_win.delete(0, tk.END)

    def open_link(self, link):
        webbrowser.open(link)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)

                # Insert bot icon and name
                self.insert_user('bot', 'Friday', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)

                    # Set the recognized text to the message attribute
                    self.message.set(text)

                    if "search" in text.lower() or "play" in text.lower():
                        # Perform search or play directly
                        self.send_msz()
                    else:
                        # Set input source to 'mic' and handle the user input
                        self.input_source = 'mic'
                        self.send_msz()

                except sr.UnknownValueError:
                    print("Friday: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"Friday: There was an error with the speech recognition service: {e}")



    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def trans(self):
        languages = google_trans_new.LANGUAGES

        # Insert bot icon and name
        self.insert_user('bot', 'Friday', self.bot_icon)

        # Check if the translation process is already ongoing
        if not hasattr(self, 'translation_in_progress'):
            # Insert bot response
            self.insert_message('bot', 'Available Languages:')
            # Insert available languages in the GUI
            for code, lang in languages.items():
                self.insert_message('bot', f'{code}: {lang}')

            mic = sr.Microphone()
            r = sr.Recognizer()

            with mic as source:
                translator = Translator()

                # Get input language from user using simpledialog
                input_language = simpledialog.askstring("Input Language", "Select Language from:")
                # Get output language from user using simpledialog
                output_language = simpledialog.askstring("Output Language", "Select your Language To:")

                self.insert_user('bot', 'Friday', self.bot_icon)
                self.insert_message('bot', 'speak the text you need to be translated..')
                r.adjust_for_ambient_noise(source, duration=0.2)
                audio3 = r.listen(source)
                text3 = r.recognize_google(audio3)
                
                self.insert_user('usr', 'You', self.user_icon)
                self.insert_message('usr',text3.lower())

                # Translate the user's question
                translated_question = translator.translate(text3, src=input_language, dest=output_language).text.lower()

                # Update GUI with the translation
                self.insert_user('bot', 'Friday', self.bot_icon)
                self.insert_message('bot', f'Translation: {translated_question}')
                if hasattr(self, 'input_source') and self.input_source == 'mic':
                    self.speak(translated_question)
                self.textcon.insert(tk.END, "\n")

                # Set the variable to indicate that translation is in progress
                self.translation_in_progress = True
                return  # Exit the function to avoid rerunning the translation process

        # If translation process is already in progress, clear the variable
        self.translation_in_progress = False


    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
C:\Users\aluru\AppData\Local\Temp\ipykernel_14140\785103868.py:63: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_14140\785103868.py:64: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_14140\785103868.py:70: Deprecation

Friday: Sorry, I could not understand the audio. Please try again.
Friday: Sorry, I could not understand the audio. Please try again.


In [11]:
# full translation not working
import tkinter as tk
from tkinter import Entry, Button
import speech_recognition as sr
import pyttsx3
import os
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import json
import threading
from datetime import datetime
import pywhatkit
from googlesearch import search
import webbrowser
from PIL import Image, ImageTk
from googletrans import Translator
from gtts import gTTS
import pygame
import google_trans_new
from tkinter import simpledialog

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Load or collect data for training/fine-tuning (replace with your actual data loading)
with open('D:/downloads/tech.json', 'r') as file:
    data = json.load(file)

class ChatbotApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title(f"NLPAssistant")
        self.root.geometry('1000x800')  # Increased dimensions
        self.root.resizable(False, False)
        self.message = tk.StringVar()
        self.is_listening = False
        self.greeted = False
        
        self.textcon = tk.Text(self.root, bd=1, width=70, height=15, wrap='word', font=('Times New Roman', 12))  # Adjusted dimensions and added wrap='word'
        self.textcon.pack(fill="both", expand=True)

        self.mes_win = Entry(self.root, width=50, xscrollcommand=True, textvariable=self.message, font=('Times New Roman', 12))
        self.mes_win.place(x=1, y=720, height=60, width=679)  # Adjusted dimensions
        self.mes_win.focus()

        self.textcon.config(fg='black')
        self.textcon.tag_config('usr', foreground='black', justify='right',font=('Times New Roman', 14, 'bold'), background='lightgreen')
        self.textcon.tag_config('bot', foreground='black', justify='left',font=('Times New Roman', 14, 'bold'), background='lightblue')

        self.textcon.insert(tk.END, "\n")

        self.exit_list = ['goodbye', 'bye', 'off']

        self.user_icon_path = "D:/downloads/user_icon.png"
        self.bot_icon_path = "D:/downloads/bot_icon.png"

        self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
        self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))

        self.mic_image_path = "D:/downloads/mic.png"

        try:
            mic_icon = Image.open(self.mic_image_path)
            mic_icon = mic_icon.resize((mic_icon.width // 2, mic_icon.height // 3), Image.ANTIALIAS)
            self.mic_image = ImageTk.PhotoImage(mic_icon)
        except Exception as e:
            print(f"Error loading mic image: {e}")
            self.mic_image = None

        self.mic_button = Button(self.root, image=self.mic_image, bg='blue', activebackground='white',
                                 command=self.activate_mic, width=12, height=2, font=('Times New Roman', 14))
        self.mic_button.place(x=800, y=720, height=60, width=110)  # Adjusted dimensions

        self.send_button = Button(self.root, text='Send', bg='cyan', activebackground='grey',
                                   command=self.send_msz, width=12, height=2, font=('Times New Roman', 14, "bold"))
        self.send_button.place(x=680, y=720, height=60, width=110)  # Adjusted dimensions

        self.root.bind('<Return>', self.send_msz)
        self.engine = pyttsx3.init()
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Add a vertical scrollbar
        scrollbar = tk.Scrollbar(self.root, command=self.textcon.yview)
        scrollbar.pack(side="right", fill="y")
        # Configure the text widget to use the scrollbar
        self.textcon.config(yscrollcommand=scrollbar.set)

        # Start the greeting in a separate thread
        threading.Thread(target=self.greet_user).start()

        self.root.mainloop()

    def greet_user(self):
        if not self.greeted:
            self.greeted = True
            # Insert bot icon and name for greeting
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response for greeting
            self.insert_message('bot', "Hi! I am Friday, your assistant.")
            current_time = datetime.now().strftime("%I:%M %p")
            # Insert additional bot response
            self.insert_message('bot', f"The current time is {current_time}")
            # Speak the greeting
            self.speak("Hello sir! I am Friday.")
            self.speak(f"The current time is {current_time}")

    def send_msz(self, event=None):
        usr_input = self.message.get()
        usr_input = usr_input.lower()

        # Insert user icon and name
        self.insert_user('usr', 'You', self.user_icon)
        # Insert user input
        self.insert_message('usr', f'{usr_input}')

        if usr_input.lower() in ["goodbye", "bye", "off"]:
            response = "Thank You sir, I hope I assisted you properly"
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.is_listening = False  # Stop the microphone listening thread
            return self.root.destroy()

        elif "play" in usr_input.lower():
            # Open YouTube using pywhatkit
            pywhatkit.playonyt(usr_input.replace("play", "").strip())
            response = f"Playing {usr_input.replace('play', '').strip()} on YouTube..."
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            self.speak(response)
            self.mes_win.delete(0, tk.END)

        elif "search" in usr_input.lower():
            search_query = usr_input.replace("search", "").strip()
            self.perform_search(search_query)

        elif "translate" in usr_input or "language" in usr_input:
            self.trans()
            self.mes_win.delete(0, tk.END)

        else:
            response = self.handle_user_input(usr_input)
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}')
            # Speak only if the input is from the microphone
            if hasattr(self, 'input_source') and self.input_source == 'mic':
                self.speak(response)
            self.mes_win.delete(0, tk.END)

    def insert_user(self, tag, name, icon):
        # Insert icon
        self.textcon.image_create(tk.END, image=icon)
        # Insert name
        self.textcon.insert(tk.END, f'{name}: ')

    def insert_message(self, tag, message):
        # Insert user or bot message with the specified tag
        self.textcon.insert(tk.END, message, tag)
        self.textcon.insert(tk.END, "\n\n")  # Add an empty line between messages
        self.textcon.see(tk.END)
        
    def perform_search(self, search_query):
        try:
            search_results = search(search_query, num=1, stop=1, pause=2)
            first_result = next(search_results)

            # Insert bot icon and name
            self.insert_user('bot', 'Friday: here is the link', self.bot_icon)
            self.textcon.tag_config('link', foreground='blue', underline=True)
            self.textcon.insert(tk.END, f'{first_result}\n', 'link')
            self.textcon.tag_bind('link', '<Button-1>', lambda event, link=first_result: self.open_link(link))
            self.textcon.insert(tk.END, "\n")
        except StopIteration:
            response = f"Sorry, I couldn't find information about {search_query}."
            # Insert bot icon and name
            self.insert_user('bot', 'Friday', self.bot_icon)
            # Insert bot response
            self.insert_message('bot', f'{response}\n')

        self.mes_win.delete(0, tk.END)

    def open_link(self, link):
        webbrowser.open(link)

    def activate_mic(self):
        if not self.is_listening:
            if self.mic_image:
                self.mic_button.config(image=self.mic_image)
            else:
                self.mic_button.config(text="Mic")

            self.is_listening = True
            threading.Thread(target=self.listen_continuously).start()

    def listen_continuously(self):
        r = sr.Recognizer()

        while self.is_listening:
            engine = pyttsx3.init()
            rate = engine.getProperty('rate')
            engine.setProperty('rate', 170)
            voices = engine.getProperty('voices')
            engine.setProperty('voice', voices[1].id)

            with sr.Microphone() as source:
                r.energy_threshold = 400
                r.adjust_for_ambient_noise(source, 1.2)

                # Insert bot icon and name
                self.insert_user('bot', 'Friday', self.bot_icon)
                # Insert bot response
                self.insert_message('bot', "Listening...")

                try:
                    audio = r.listen(source)
                    text = r.recognize_google(audio)

                    # Set the recognized text to the message attribute
                    self.message.set(text)

                    if "search" in text.lower() or "play" in text.lower() or "translate" in text.lower():
                        # Perform search or play directly
                        self.send_msz()
                    else:
                        # Set input source to 'mic' and handle the user input
                        self.input_source = 'mic'
                        self.send_msz()

                except sr.UnknownValueError:
                    print("Friday: Sorry, I could not understand the audio. Please try again.")

                except sr.RequestError as e:
                    print(f"Friday: There was an error with the speech recognition service: {e}")



    def speak(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

    def handle_user_input(self, user_input):
        user_keywords = self.extract_keywords(user_input)

        for example in data.get("intents", []):
            if "question" in example:
                question_keywords = [keyword for q in example["question"] for keyword in self.extract_keywords(q)]

                if all(keyword in question_keywords for keyword in user_keywords):
                    answer = example.get("answer", [])
                    return f"{answer}"

        input_ids = tokenizer.encode(user_input, return_tensors="tf")
        output = gpt_model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95,
                                    temperature=0.7)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Adjust the response based on the user's technical ability
        technical_ability = "tech"  # You need to get this value from somewhere
        if technical_ability == "tech":
            # Customize response for technical users
            pass
        else:
            # Customize response for non-technical users
            pass

        candidate_answers = ["Your first answer", "Your second answer", "Your third answer"]
        relevancy_scores = self.score_relevancy(user_input, candidate_answers)

        # Store or update relevancy scores for future learning
        # Update your model based on user feedback and learning algorithm

        return bot_response

    def extract_keywords(self, text):
        doc = nlp(text)
        return [token.text.lower() for token in doc if token.is_alpha]

    def score_relevancy(self, user_input, candidate_answers):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([user_input] + candidate_answers)
        similarity_matrix = cosine_similarity(vectors)
        relevancy_scores = similarity_matrix[0][1:]
        return relevancy_scores

    def trans(self):
        languages = google_trans_new.LANGUAGES

        # Insert bot icon and name
        self.insert_user('bot', 'Friday', self.bot_icon)

        # Check if the translation process is already ongoing
        if not hasattr(self, 'translation_in_progress'):
            # Insert bot response
            self.insert_message('bot', 'Available Languages:')
            # Insert available languages in the GUI
            for code, lang in languages.items():
                self.insert_message('bot', f'{code}: {lang}')

            mic = sr.Microphone()
            r = sr.Recognizer()

            with mic as source:
                translator = Translator()

                # Get input language from user using simpledialog
                input_language = simpledialog.askstring("Input Language", "Select Language from:")
                # Get output language from user using simpledialog
                output_language = simpledialog.askstring("Output Language", "Select your Language To:")

                self.insert_user('bot', 'Friday', self.bot_icon)
                self.insert_message('bot', 'speak the text you need to be translated..')
                r.adjust_for_ambient_noise(source, duration=0.2)
                audio3 = r.listen(source)
                text3 = r.recognize_google(audio3)
                
                self.insert_user('usr', 'You', self.user_icon)
                self.insert_message('usr',text3.lower())

                # Translate the user's question
                translated_question = translator.translate(text3, src=input_language, dest=output_language).text.lower()

                # Update GUI with the translation
                self.insert_user('bot', 'Friday', self.bot_icon)
                self.insert_message('bot', f'Translation: {translated_question}')
                if hasattr(self, 'input_source') and self.input_source == 'mic':
                    self.speak(translated_question)
                self.textcon.insert(tk.END, "\n")

                # Set the variable to indicate that translation is in progress
                self.translation_in_progress = True
                return  # Exit the function to avoid rerunning the translation process

        # If translation process is already in progress, clear the variable
        self.translation_in_progress = False


    def on_close(self):
        self.is_listening = False
        self.root.destroy()

if __name__ == "__main__":
    chatbot_app = ChatbotApp()


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
C:\Users\aluru\AppData\Local\Temp\ipykernel_14140\1367163038.py:63: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.user_icon = ImageTk.PhotoImage(Image.open(self.user_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_14140\1367163038.py:64: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  self.bot_icon = ImageTk.PhotoImage(Image.open(self.bot_icon_path).resize((40, 40), Image.ANTIALIAS))
C:\Users\aluru\AppData\Local\Temp\ipykernel_14140\1367163038.py:70: Deprecat